# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

作业 2.（可选）结合 DeepSeek RAG 课程中学习到的方法，将示例代码中的模拟工具 mock_query_product_database 替换为真实的产品 / 商品数据。

## 1. 环境准备与DeepSeek API配置

In [8]:
import os

from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

## 2. 利用 Milvus 加载产品数据

### 2.1 准备数据

In [22]:
# 读取 product.json 文件
import json

# 打开并读取JSON文件
with open("products.json", "r", encoding="utf-8") as f:
    product_data = json.load(f)

product_texts = [f"{p['name']}: {p['description']}" for p in product_data]

product_texts[:5]

['焕亮净透面膜: 蕴含天然酵母精华与果酸，能温和去除角质，提亮肤色，让肌肤恢复透亮感。适合暗沉粗糙肌肤。',
 '深层补水修复面膜: 采用透明质酸和神经酰胺，提供24小时锁水能力，修复受损肌肤屏障，缓解干燥紧绷。适合干性缺水肌肤。',
 '舒缓镇静面膜: 含有积雪草提取物和芦荟胶，能快速舒缓泛红、敏感肌肤，减轻外界刺激带来的不适。适合敏感肌日常护理。',
 '紧致抗皱面膜: 核心成分是胜肽和胶原蛋白，能促进胶原蛋白生成，提升肌肤弹性，减少细纹和皱纹。适合熟龄肌肤。',
 '控油清爽面膜: 含有茶树精油和火山泥，能深入毛孔吸附多余油脂，净化肌肤，收缩毛孔。适合油性及混合性肌肤。']

### 2.2 准备 Embedding 模型

In [74]:
# from pymilvus import model

# embedding_model = model.dense.Model2VecEmbeddingFunction(
#     model_source="minishlab/potion-retrieval-32M",
# )
from pymilvus.model.hybrid import MGTEEmbeddingFunction

embedding_model = MGTEEmbeddingFunction(
    # model_name="Alibaba-NLP/gte-multilingual-base",  # Defaults to `Alibaba-NLP/gte-multilingual-base`
    model_name="../../../local_models/gte-multilingual-base",
)


生成一个测试嵌入并打印其维度和前几个元素。

In [75]:
test_embedding = embedding_model.encode_queries(["面膜"])

print(embedding_model.dim["dense"])
print(test_embedding["dense"][0].tolist()[:10])

768
[-0.02401237189769745, -0.01778465323150158, 0.0014521379489451647, -0.01873784139752388, 0.052740518003702164, -0.03842202201485634, -0.048756204545497894, 0.04386601224541664, 0.058112602680921555, -0.056739214807748795]


### 2.3 加载数据

In [48]:
from pymilvus import MilvusClient
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 添加这行禁用tokenizers并行
milvus_client = MilvusClient(uri="./milvus.db")

In [49]:
collection_name = "facial_mask_collection"
# 检查 collection 是否已存在，如果存在则删除它。
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_model.dim["dense"],
    metric_type="IP",  # 内积距离
    consistency_level="Strong",
    # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。
    # 更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [54]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(product_texts)["dense"]

for i, line in enumerate(tqdm(product_texts, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i].tolist(), "text": line})

print(data[0]["vector"][:5], data[0]["text"])
print(data[1]["vector"][:5], data[1]["text"])

Creating embeddings: 100%|██████████| 51/51 [00:00<00:00, 4630.98it/s]

[0.02950448729097843, 0.03165130689740181, 0.016492702066898346, 0.004975141957402229, 0.05252726376056671] 焕亮净透面膜: 蕴含天然酵母精华与果酸，能温和去除角质，提亮肤色，让肌肤恢复透亮感。适合暗沉粗糙肌肤。
[0.0040726629085838795, -0.03283930942416191, -0.002429644111543894, 0.025488583371043205, 0.02100597508251667] 深层补水修复面膜: 采用透明质酸和神经酰胺，提供24小时锁水能力，修复受损肌肤屏障，缓解干燥紧绷。适合干性缺水肌肤。


In [55]:
milvus_client.insert(collection_name=collection_name, data=data)

{'insert_count': 51, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], 'cost': 0}

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [23]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用 'Thought-Action-Observation' 模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [25]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'",
                    }
                },
                "required": ["product_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'",
                    }
                },
                "required": ["context"],
            },
        },
    },
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

其中，`query_product_database` 从向量数据库查询产品数据


In [61]:
import random  # 用于模拟生成表情
import time  # 用于模拟网络延迟


def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1)  # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"


def query_product_database(product_name: str) -> str:
    """查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 查询产品数据库：{product_name}")

    encoded_data = embedding_model.encode_queries([product_name])
    data = [d.tolist() for d in encoded_data["dense"]]
    res = milvus_client.search(
        collection_name=collection_name,
        data=data,  # 将问题转换为嵌入向量
        limit=5,  # 返回前5个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )
    retrieved_texts = [r["entity"]["text"] for r in res[0]]
    for txt in retrieved_texts:
        if product_name in txt:
            return txt
    return retrieved_texts[0]


def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2)  # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(
            ["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"],
            k=min(5, len(context.split())),
        )


# 将模拟工具函数映射到字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [64]:
# 测试查询
query_product_database("薰衣草修复面膜")

[Tool Call] 查询产品数据库：薰衣草修复面膜


'薰衣草修复面膜: 含有薰衣草精油和修复成分，舒缓敏感肌肤，促进肌肤修复，带来健康肌肤。适合敏感肌。'

## 4. 实战：构建小红书文案生成 Agent

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [65]:
import json


def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", [])

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n"  # 标题使用二级标题

    # 正文，保留换行符
    markdown_output += f"{body}\n\n"

    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags)  # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"

    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"

    return markdown_output.strip()  # 去除末尾多余的空白

In [66]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = (
    "{'title': 'Test', 'body': 'This is not valid json'}"  # 使用单引号，非法
)
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


### generate_rednote

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [67]:
import re


def generate_rednote(
    product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5
) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。

    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。

    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。",
        },
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")

        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION,  # 告知模型可用的工具
                tool_choice="auto",  # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message

            # **ReAct模式：处理工具调用**
            if response_message.tool_calls:  # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message)  # 将工具调用信息添加到对话历史

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = (
                        json.loads(tool_call.function.arguments)
                        if tool_call.function.arguments
                        else {}
                    )

                    print(
                        f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}"
                    )

                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append(
                            {
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "content": str(tool_result),  # 工具结果作为字符串返回
                            }
                        )
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append(
                            {
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "content": error_message,
                            }
                        )
                messages.extend(
                    tool_outputs
                )  # 将工具执行结果作为 Observation 添加到对话历史

            # **ReAct 模式：处理最终内容**
            elif response_message.content:  # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")

                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(
                    r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL
                )

                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message)  # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print(
                            "Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。"
                        )
                        messages.append(response_message)  # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break

        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break

    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [70]:
# 测试案例 1: 黑炭深层清洁面膜
product_name_1 = "玫瑰保湿面膜"
tone_style_1 = "幽默风趣"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1  (Markdown)---")
print(format_rednote_for_markdown(result_1))


🚀 启动小红书文案生成助手，产品：玫瑰保湿面膜，风格：幽默风趣

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '玫瑰保湿面膜'}
[Tool Call] 查询产品数据库：玫瑰保湿面膜
Observation: 工具返回结果：玫瑰保湿面膜: 蕴含大马士革玫瑰精油和透明质酸，深层补水，改善肌肤干燥，令肌肤水润有光泽。适合干性及缺水肌肤。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '玫瑰保湿面膜，补水保湿，幽默风趣'}
[Tool Call] 模拟生成表情符号，上下文：玫瑰保湿面膜，补水保湿，幽默风趣
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "🌹玫瑰保湿面膜｜干皮救星！用完脸嫩到能掐出水💦",
  "body": "姐妹们！我又挖到宝了！这款玫瑰保湿面膜简直是干皮的亲妈！🌹\n\n最近换季，我的脸干到能刮出粉来😭，直到我遇到了它——玫瑰保湿面膜！\n\n✨ 成分党看过来：大马士革玫瑰精油+透明质酸，双重补水，敷完脸直接嫩到发光！💧\n\n🌊 使用方法超简单：洗完脸敷15分钟，揭下来那一刻，感觉皮肤在狂喝水！\n\n最搞笑的是，昨晚敷完面膜，男朋友居然问我是不是偷偷打了水光针🤣（内心OS：这面膜比男票还会夸人！）\n\n干皮姐妹闭眼冲！用完你会回来感谢我的！💖",
  "hashtags": ["#干皮救星", "#玫瑰保湿面膜", #补水神器", "#护肤好物", "#换季必备"],
  "emojis": ["🌹", "💦", "💧", "✨", "💖"]
}
```
Agent: 提取到JSON块但解析失败: Expecting value: line 4 column 36 (char 314)
尝试解析的字符串:
{
  "title": "🌹玫瑰保湿面膜｜干皮救星！用完脸嫩到能掐出水💦",
  "body": "姐妹们！我又挖到宝了！这款玫瑰保湿